In [11]:
import importlib
import os
from pathlib import Path
import sys

from dotenv import load_dotenv, find_dotenv
import pandas as pd
import requests

dir_src = Path.cwd().parent.parent.parent/'geosaurus'/'src'

sys.path.insert(0, str(dir_src))

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS, server
from arcgis import modeling
from arcgis.modeling import Modeling

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

# create a GIS object instance; if you did not enter any information here, it defaults to anonymous access to ArcGIS Online
gis_portal = GIS(
    url=os.getenv('ESRI_PORTAL_URL'), 
    username=os.getenv('ESRI_PORTAL_USERNAME'),
    password=None if len(os.getenv('ESRI_PORTAL_PASSWORD')) is 0 else os.getenv('ESRI_PORTAL_PASSWORD')
)

gis_agol = GIS(
    url=os.getenv('ESRI_GIS_URL'), 
    username=os.getenv('ESRI_GIS_USERNAME'),
    password=None if len(os.getenv('ESRI_GIS_PASSWORD')) is 0 else os.getenv('ESRI_GIS_PASSWORD')
)

gis_agol

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


GIS @ https://geoai.maps.arcgis.com version:8.4

In [2]:
usa_prtl = modeling.Country('USA', source=gis_portal)

usa_prtl.levels

,geo_name,geo_alias,admin_level,singular_name,plural_name,id
0,block_groups,Block Groups,,Block Group,Block Groups,US.BlockGroups
1,census_tracts,Census Tracts,,Census Tract,Census Tracts,US.Tracts
2,cities_and_towns_places,Cities and Towns (Places),,Place,Places,US.Places
3,zip_codes,ZIP Codes,Admin4,ZIP Code,ZIP Codes,US.ZIP5
4,county_subdivisions,County Subdivisions,,County Subdivision,County Subdivisions,US.CSD
5,counties,Counties,Admin3,County,Counties,US.Counties
6,cbsas,CBSAs,,CBSA,CBSAs,US.CBSA
7,congressional_districts,Congressional Districts,,Congressional District,Congressional Districts,US.CD
8,dmas,DMAs,,DMA,DMAs,US.DMA
9,states,States,Admin2,State,States,US.States


In [3]:
usa = modeling.Country('USA')

In [4]:
usa.enrich_variables

,name,alias,data_collection,enrich_name,enrich_field_name
0,AGE0_CY,2020 Population Age <1,1yearincrements,1yearincrements.AGE0_CY,F1yearincrements_AGE0_CY
1,AGE1_CY,2020 Population Age 1,1yearincrements,1yearincrements.AGE1_CY,F1yearincrements_AGE1_CY
2,AGE2_CY,2020 Population Age 2,1yearincrements,1yearincrements.AGE2_CY,F1yearincrements_AGE2_CY
3,AGE3_CY,2020 Population Age 3,1yearincrements,1yearincrements.AGE3_CY,F1yearincrements_AGE3_CY
4,AGE4_CY,2020 Population Age 4,1yearincrements,1yearincrements.AGE4_CY,F1yearincrements_AGE4_CY
...,...,...,...,...,...
9222,ACSRMV2000,2018 RHHs/Moved In: 2000-2009 (ACS 5-Yr),yearmovedin,yearmovedin.ACSRMV2000,yearmovedin_ACSRMV2000
9223,ACSRMV1990,2018 RHHs/Moved In: 1990-1999 (ACS 5-Yr),yearmovedin,yearmovedin.ACSRMV1990,yearmovedin_ACSRMV1990
9224,ACSRMV1989,2018 RHHs/Moved In: 1989/Before (ACS 5-Yr),yearmovedin,yearmovedin.ACSRMV1989,yearmovedin_ACSRMV1989
9225,ACSMEDYRMV,2018 Median Year Householder Moved In (ACS 5-Yr),yearmovedin,yearmovedin.ACSMEDYRMV,yearmovedin_ACSMEDYRMV


In [16]:
cnty_df = usa.states.get('Washington').modeling.counties.get('Thurston')

cnty_df

,ID,NAME,SHAPE
0,53067,Thurston County,"{""rings"": [[[-122.91039900289185, 47.045107999..."


In [17]:
bg_df = cnty_df.modeling.block_groups.get()

bg_df

,ID,NAME,SHAPE
0,530670101001,530670101.001,"{""rings"": [[[-122.89850000038508, 47.056410001..."
1,530670101002,530670101.002,"{""rings"": [[[-122.89252199963018, 47.043792999..."
2,530670101003,530670101.003,"{""rings"": [[[-122.90402399978194, 47.043902999..."
3,530670101004,530670101.004,"{""rings"": [[[-122.8958470004666, 47.0380299995..."
4,530670102001,530670102.001,"{""rings"": [[[-122.88865899956669, 47.076915999..."
...,...,...,...
154,530670122211,530670122.211,"{""rings"": [[[-122.81004105640477, 47.161288078..."
155,530670122212,530670122.212,"{""rings"": [[[-122.81772499920675, 47.130394999..."
156,530670122213,530670122.213,"{""rings"": [[[-122.81132999959863, 47.086874000..."
157,530670122221,530670122.221,"{""rings"": [[[-122.78801999987161, 47.132557025..."


In [19]:
modeling.get_countries(gis_agol)

,iso2,iso3,country_name,country_id,alt_name,continent
0,AL,ALB,Albania,ALB_MBR_2019,ALBANIA,Europe
1,DZ,DZA,Algeria,DZA_MBR_2019,ALGERIA,Africa
2,AD,AND,Andorra,AND_MBR_2019,ANDORRA,Europe
3,AO,AGO,Angola,AGO_MBR_2019,ANGOLA,Africa
4,AR,ARG,Argentina,ARG_MBR_2018,ARGENTINA,South America
...,...,...,...,...,...,...
131,UY,URY,Uruguay,URY_MBR_2018,URUGUAY,South America
132,UZ,UZB,Uzbekistan,UZB_MBR_2020,UZBEKISTAN,Asia
133,VE,VEN,Venezuela,VEN_MBR_2018,"VENEZUELA, BOLIVARIAN REPUBLIC OF",South America
134,VN,VNM,Vietnam,VNM_MBR_2020,VIET NAM,Asia


In [21]:
uzb = modeling.Country('UZB', gis_agol)

uzb

<modeling.Country - UZB (GIS at https://geoai.maps.arcgis.com logged in as jmccune_geoai)>

In [23]:
uzb.levels

,geo_name,geo_alias,admin_level,singular_name,plural_name,id
0,provinces,Provinces,Admin2,Province,Provinces,UZ.Provinces
1,country,Country,Admin1,Country,Country,UZ.Country
